We will start the InfluxDB and Grafana containers using the makefile that we have in the project. 

In [1]:
!make -f ../Makefile run INFLUXDB_DB=gpudata INFLUXDB_USER=admin INFLUXDB_USER_PASSWORD=password GF_ADMIN_PASSWORD=secret

docker run -d -p 8083:8083 -p 8086:8086 \
-e INFLUXDB_DB=gpudata \
-e INFLUXDB_USER=admin \
-e INFLUXDB_USER_PASSWORD=password \
-v /home/mat/repos/influx_grafana/examples:/var/lib/influxdb \
-e INFLUXDB_ADMIN_ENABLED=true \
--name influxdb \
influxdb
echo "Influx DB gpudata set up waiting on port 8083"
2286c8d0ed3bd46e0426dd33712a01c0aacfd8541ae1613e861b65b50c5ea7b0
Influx DB gpudata set up waiting on port 8083
# create /var/lib/grafana as persistent volume storage
docker run -d -v /var/lib/grafana --name grafana-storage busybox:latest
# start grafana
docker run \
  -d \
  -p 3000:3000 \
  --name=grafana \
  --volumes-from grafana-storage \
  --link influxdb:influxdb \
  -e GF_SECURITY_ADMIN_PASSWORD=secret \
  grafana/grafana
echo "Grafana set up and waiting on port 3000"
9c68cc8dbbb79c6b5f9f1b57c5f1fc5dd8518031e9ade0dda6beb9e747c9154b
5b4229935525f7cf68c5218d8cc2225cdca7a24d824bdc1a2ab055a2f9f66235
Grafana set up and waiting on port 3000
InfluxDB and Grafana started
Influx DB gpudat

In [2]:
from influxdb import InfluxDBClient, DataFrameClient
from datetime import datetime

Below we will simply create some fake measurement data and write them to the database gpudata. The measurement variable is what is used to define the measurement/series/table. If we do not pass time then influxdb will simply enter its own timestamp. Tags can be used to group or select timeseries.

In [3]:
json_body = [
    {
        "measurement": "gpu",
        "tags": {
            "gpu": "0",
            "machine": "my_machine"
        },
        "time": str(datetime.today()),
        "fields": {
            "value": 0.64
        }
    }
]

In [4]:
client = InfluxDBClient('localhost', 8086, 'admin', 'password', 'gpudata')

In [5]:
client.get_list_database()

[{'name': 'gpudata'}]

In [5]:
client.write_points(json_body)

True

In [6]:
result = client.query('select * from gpu;')

In [7]:
print("Result: {0}".format(result))

Result: ResultSet({'('gpu', None)': [{'time': '2017-12-12T12:20:19.09255296Z', 'gpu': '0', 'machine': 'my_machine', 'value': 0.64}]})


In [8]:
client.get_list_database()

[{'name': 'gpudata'}]

In [9]:
client.get_list_measurements()

[{'name': 'gpu'}]

In [10]:
client.drop_measurement('gpu')

In [11]:
client.get_list_measurements()

[]

In [12]:
client.get_list_database()

[{'name': 'gpudata'}, {'name': '_internal'}]

To clean everything up we just invoke the makefile clean command. This will stop and remove the containers and delete the data.

In [13]:
!sudo make -f ../Makefile clean

docker stop influxdb grafana-storage grafana
influxdb
grafana-storage
grafana
rm -r data meta wal
docker rm -v influxdb grafana-storage grafana
influxdb
grafana-storage
grafana
